# cppyy tutorial
#### all credit to the author, Wim Lavrijsen
#### bugs and feature requests at https://bitbucket.org/wlav/cppyy/issues

![title](./cpp_python_feature_mapping.png)

In [ ]:
from __future__ import print_function
p = print
import cppyy

## Define a class

In [ ]:
cppyy.cppdef("""
  struct Integer {
    Integer(int i) : num(i) {}
    int num;
    private:
    int p_num = -1;
  };
""")
from cppyy.gbl import Integer

In [ ]:
m1 = Integer(42)

p(m1.num)
try:
    p(m1.p_num)
except:
    p('p_num not available')

### Autogenerated help/doc

In [ ]:
Integer?

In [ ]:
help(Integer)

## Pythonizations: dynamically augment python bindings with methods/attributes

In [ ]:
cppyy.cppdef("""
    Integer add(Integer &m1, Integer &m2) {
        return Integer(m1.num + m2.num);
    }
""")
Integer.__add__ = cppyy.gbl.add # addition from a c++ function
Integer.__repr__ = lambda i : "integer: " + str(i.num) # string representation as python lambda

In [ ]:
p(Integer(3) + Integer(39))

## Inheritance

In [ ]:
cppyy.cppdef("""
   struct Base {
       virtual std::string method() { return "Base"; }
   };
   
   struct Derived : Base {
       std::string method() override { return "Derived"; }
   };""")
Base, Derived = cppyy.gbl.Base, cppyy.gbl.Derived

In [ ]:
p(Base().method())
p(Derived().method())
issubclass(Derived, Base)

## Operator overloading

In [ ]:
cppyy.cppdef("""
    struct IntValue {
        IntValue(int v) : num(v) {}
        IntValue &operator+(const IntValue &i) { num += i.num; return *this; }
        IntValue &operator+(int i) { num += i; return *this; }
        int num;
    };""")
IntValue = cppyy.gbl.IntValue

In [ ]:
IntValue.__repr__ = lambda self : str(self.num)

p(IntValue(1) + IntValue(1))
p(IntValue(2**31-1) + IntValue(2**31-1)) # overflow happens in C++, not checked for
try:
    (IntValue(2**33))
except:
    
x, y, z = IntValue(1), IntValue(2), IntValue(3)
p(x + y + z)

## Template classes

In [ ]:
vector = cppyy.gbl.vector
v = vector("unsigned short")((0, 8, 42)) # vector[int] and vector("int") also allowed
print(v[0], v[1], v[2])

In [ ]:
cppyy.cppdef("""
template <typename T, int N>
struct fac
{
    static constexpr int value = N * fac<T, N-1>::value;
};

template <typename T>
struct fac<T, 0>
{
    static constexpr int value = 1;
};
""")
from cppyy.gbl import fac

In [ ]:
%time print(fac(int, 5).value)

## Function overloading

In [ ]:
cppyy.cppdef("""
double dub(double x) { return x*2; }

int dub(int x) { return x*2; }

template <typename T>
T dub(T t) { return t + t; }
""")
dub = cppyy.gbl.dub

In [ ]:
type(dub(3))

In [ ]:
type(dub(3.))

In [ ]:
i = IntValue(2)
i2 = dub(i)
print(i2, '---', type(i2))

In [ ]:
dub(cppyy.gbl.std.string("a"))

In [ ]:
dub("a")

## Template methods

In [ ]:
cppyy.cppdef("""
struct Doubler {
    int operator()(int t) { return t + t; } // at least one overload must be non-template

    template <typename T>
    T operator()(T t) { return t + t; }
};
""")
from cppyy.gbl import Doubler

In [ ]:
dub = Doubler()
six = dub(IntValue(3))
print(six, '---', type(six))

## Lambdas

In [ ]:
cppyy.cppdef("""
auto make_lambda() {
    int b = 40;
    return [&b](int a) { return b+a; };
}
auto l = make_lambda();
""")
print(cppyy.gbl.l(2))

## Including a header

In [ ]:
%%file /tmp/cppy_test.h
std::string bar() { return "oh yeah"; }    

In [ ]:
cppyy.include('/tmp/cppyy_test.h')

In [ ]:
p(cppyy.gbl.bar())

In [ ]:
# cppyy.load_library to load the corresponding shared object

## Functions that take a pointer

In [ ]:
cppyy.cppdef("""
int GetNum(IntValue *v) { return v->num; }
""")

In [ ]:
cppyy.cppdef("""
int GetNum(IntValue &v) { return v.num + 5; }
""")

In [ ]:
cppyy.gbl.GetNum(IntValue(3))

## TTree to numpy: pythonizations

custom python module augments ROOT classes when imported: `$ROOTSYS/ROOT.py`, e.g.:

```python
# a python function that retrieves large datasets via a C++ helper, then converts to numpy array
def _TTreeAsMatrix():
    ...
    pass

# actual type augmentation
_root.CreateScopeProxy('TTree').AsMatrix = _TTreeAsMatrix
```

## Unsupported: template overload resolution

In [ ]:
import cppyy

cppyy.cppdef("""
struct Obj {
    int var;
};

template <typename T>
constexpr auto has_var(T t) -> decltype(t.var, true) { return true; }

template <typename ...Args>
constexpr bool has_var(Args...) { return false; }

template <typename T>
constexpr bool call_has_var(T &&t) { return has_var(std::forward<T>(t)); }

void foo() {
    static_assert(has_var(Obj()), "");
    static_assert(!has_var(3), "");
}
""")

Obj = cppyy.gbl.Obj

cppyy.gbl.has_var(Obj()) # doesn't work

cppyy.gbl.call_has_var("Obj")(Obj()) # works